# Exercise 1: Using Joins

## Question 1: 
Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98.

```
SELECT cps.NAME_OF_SCHOOL, cps.COMMUNITY_AREA_NAME, cps.AVERAGE_STUDENT_ATTENDANCE
FROM chicago_public_schools cps
LEFT JOIN chicago_socioeconomic_data csd
ON cps.COMMUNITY_AREA_NUMBER = csd.COMMUNITY_AREA_NUMBER
WHERE csd.HARDSHIP_INDEX = 98
```

![Alt text](question1.1.png)

## Question 2:

Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name.

```
SELECT cc.CASE_NUMBER, cc.PRIMARY_TYPE, csd.COMMUNITY_AREA_NAME
FROM chicago_crime cc
LEFT JOIN chicago_socioeconomic_data csd
ON cc.COMMUNITY_AREA_NUMBER = csd.COMMUNITY_AREA_NUMBER
WHERE cc.LOCATION_DESCRIPTION LIKE '%SCHOOL%'
```

![Alt text](question1.2.png)

# Exercise 2: Creating a View

# Question 1

Write and execute a SQL statement to create a view showing the columns listed in the following table, with new column names as shown in the second column.

| Column name in CHICAGO_PUBLIC_SCHOOLS | Column name in view |
|---------------------------------------|---------------------|
| NAME_OF_SCHOOL                        | School_Name         |
| Safety_Icon                           | Safety_Rating       |
| Family_Involvement_Icon               | Family_Rating       |
| Environment_Icon                      | Environment_Rating  |
| Instruction_Icon                      | Instruction_Rating  |
| Leaders_Icon                          | Leaders_Rating      |
| Teachers_Icon                         | Teachers_Rating     |

* Write and execute a SQL statement that returns all of the columns from the view.

* Write and execute a SQL statement that returns just the school name and leaders rating from the view.

```
CREATE VIEW list_chicago_school AS
SELECT
    NAME_OF_SCHOOL AS School_Name,
    Safety_Icon	AS Safety_Rating,
    Family_Involvement_Icon	AS Family_Rating,
    Environment_Icon AS Environment_Rating,
    Instruction_Icon AS Instruction_Rating,
    Leaders_Icon AS Leaders_Rating,
    Teachers_Icon AS Teachers_Rating
FROM chicago_public_schools;

SELECT School_Name, Leaders_Rating
FROM list_chicago_school;
``

![Alt text](question2.1.png)

# Exercise 3: Creating a Stored Procedure

## Question 1

Write the structure of a query to create or replace a stored procedure called UPDATE_LEADERS_SCORE that takes a in_School_ID parameter as an integer and a in_Leader_Score parameter as an integer.

```
--#SET TERMINATOR @ 
CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE(
	IN in_School_ID INTEGER, 
	IN in_Leader_Score INTEGER)
LANGUAGE SQL
READS SQL DATA

DYNAMIC RESULT SETS 1
BEGIN

END @
```

![Alt text](question3.1.png)

## Question 2
Inside your stored procedure, write a SQL statement to update the Leaders_Score field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID to the value in the in_Leader_Score parameter.

```
--#SET TERMINATOR @ 
CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE(
	IN in_School_ID INTEGER, 
	IN in_Leader_Score INTEGER)
LANGUAGE SQL
MODIFIES SQL DATA

DYNAMIC RESULT SETS 1
BEGIN
	UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Score = in_Leader_Score
    WHERE School_ID = in_School_ID;
END @
```

![Alt text](question3.2.png)

## Question 3

Inside your stored procedure, write a SQL IF statement to update the Leaders_Icon field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID using the following information.

| **Score lower limit** | **Score upper limit** | **Icon**    |
|-----------------------|-----------------------|-------------|
| 80                    | 99                    | Very strong |
| 60                    | 79                    | Strong      |
| 40                    | 59                    | Average     |
| 20                    | 39                    | Weak        |
| 0                     | 19                    | Very weak   |

```
--#SET TERMINATOR @ 
CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE(
	IN in_School_ID INTEGER, 
	IN in_Leader_Score INTEGER)
LANGUAGE SQL
MODIFIES SQL DATA

BEGIN 
	UPDATE CHICAGO_PUBLIC_SCHOOLS
	SET "Leaders_Score" = in_Leader_Score
	WHERE "School_ID" = in_School_ID;
			
	IF in_Leader_Score > 0 AND in_Leader_Score < 20 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Very Weak';
	ELSEIF in_Leader_Score < 40 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Weak';	
	ELSEIF in_Leader_Score < 60 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Average';
	ELSEIF in_Leader_Score < 80 THEN
		UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Strong';
	ELSEIF in_Leader_Score < 100 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Very Strong';
	END IF;
END @
```

![Alt text](question3.3.png)

## Question 4
Run your code to create the stored procedure.

![Alt text](question3.4.png)

# Exercise 4: Using Transactions
You realise that if someone calls your code with a score outside of the allowed range (0-99), then the score will be updated with the invalid data and the icon will remain at its previous value. There are various ways to avoid this problem, one of which is using a transaction.

## Question 1
Update your stored procedure definition. Add a generic ELSE clause to the IF statement that rolls back the current work if the score did not fit any of the preceding categories.

```
--#SET TERMINATOR @ 
CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE(
	IN in_School_ID INTEGER, 
	IN in_Leader_Score INTEGER)
LANGUAGE SQL
MODIFIES SQL DATA

BEGIN 
	UPDATE CHICAGO_PUBLIC_SCHOOLS
	SET "Leaders_Score" = in_Leader_Score
	WHERE "School_ID" = in_School_ID;
			
	IF in_Leader_Score > 0 AND in_Leader_Score < 20 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Very Weak';
	ELSEIF in_Leader_Score < 40 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Weak';	
	ELSEIF in_Leader_Score < 60 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Average';
	ELSEIF in_Leader_Score < 80 THEN
		UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Strong';
	ELSEIF in_Leader_Score < 100 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Very Strong';
	ELSE
		ROLLBACK WORK;
	END IF;
END @
```

![Alt text](question4.1.png)

## Question 2
Update your stored procedure definition again. Add a statement to commit the current unit of work at the end of the procedure.

```
--#SET TERMINATOR @ 
CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE(
	IN in_School_ID INTEGER, 
	IN in_Leader_Score INTEGER)
LANGUAGE SQL
MODIFIES SQL DATA

BEGIN 
	UPDATE CHICAGO_PUBLIC_SCHOOLS
	SET "Leaders_Score" = in_Leader_Score
	WHERE "School_ID" = in_School_ID;
			
	IF in_Leader_Score > 0 AND in_Leader_Score < 20 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Very Weak';
	ELSEIF in_Leader_Score < 40 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Weak';	
	ELSEIF in_Leader_Score < 60 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Average';
	ELSEIF in_Leader_Score < 80 THEN
		UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Strong';
	ELSEIF in_Leader_Score < 100 THEN
	    UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Icon" = 'Very Strong';
	ELSE
		ROLLBACK WORK;
	END IF;
		COMMIT WORK;
END @
```

![Alt text](question4.2.png)